In [196]:
import pandas as pd
import re
import PyPDF2
import os
from os import walk
import os.path
from os import path

import time
from datetime import date

In [197]:
jurs = [["MI","Michigan"],["UT","Utah"]]
default_jur = "MI"

today = date.today().strftime("%Y-%m-%d")

In [198]:
files_df = pd.read_csv("docs/forms/form_data.csv")
files_df

,id,jurisdiction,source,title,group,url,filename,downloaded,pages,fields,f_per_p,reading
0,ed1efd08e3304bdaa887db753509d666,UT,www.utcourts.gov,Community Service Worksheet Third District Juv...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,COMMUNITY%20SERVICE%20WORKSHEET-FRONT%20AND%20...,2021-11-03,2,[],0.000000,10.0
1,28ab1fb8df5c477cb4353e44f6d5a404,UT,www.utcourts.gov,Third District Juvenile Court Work Program Ref...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,Work_Crew_Application-2007.pdf,2021-11-03,2,[],0.000000,11.0
2,279083cdac044d22b873a6a6ab8d9a14,UT,www.utcourts.gov,Utah State District Juvenile Court Probation O...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,Probation%20Order%20revised.050502.pdf,2021-11-03,5,[],0.000000,15.0
3,ebad2caa6be0458d9e5c75635a7f66c2,UT,www.utcourts.gov,Adoptee's Consent to Adoption and Waiver of Ri...,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,02_Consent_Adoptee.pdf,2021-11-03,3,[],0.000000,10.0
4,92932e4f938440ce8b93b6434aedfc3d,UT,www.utcourts.gov,Adoption Agreement,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,08_Agreement.pdf,2021-11-03,1,[],0.000000,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...
75,2ae35d7dfa7e465c99736a9740fd4560,MI,www.courts.michigan.gov,Judgment of Sentence/Commitment to Department ...,juvenile designated forms,https://www.courts.michigan.gov/4a7df7/siteass...,jc72.pdf,2021-11-03,2,['STATE OF MICHIGAN JUDICIAL CIRCUIT FAMILY D...,52.500000,8.0
76,fd9424f053a44e8280befd0d365c6ca1,MI,www.courts.michigan.gov,Order after Hearing on Petition to Accept Rele...,safe delivery newborn,https://www.courts.michigan.gov/4a2dc0/siteass...,ccfd08.pdf,2021-11-03,1,"['cirno', 'county', 'caseno', 'cname', 'judge'...",26.000000,26.0
77,fa8c956052f94a78a0c951328006bd62,MI,www.courts.michigan.gov,Case Evaluator Application,case evaluation,https://www.courts.michigan.gov/4a2af9/siteass...,mc34.pdf,2021-11-03,3,"['raddress', 'htelno', 'baddress', 'btelno', '...",25.333333,22.0
78,1ceaeb87496540e1b0b46929498536fd,MI,www.courts.michigan.gov,Collecting Your Money from a Small Claims Judg...,small claims,https://www.courts.michigan.gov/4a801f/siteass...,dci84.pdf,2021-11-03,2,[],0.000000,9.0


In [199]:
def header(title,description,path="./",wide=0):
    
    if wide==1:
        pclass = "content_wide"
    else:
        pclass = "content"
    
    html = """<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<HTML xmlns="http://www.w3.org/1999/xhtml"
      xmlns:og="http://ogp.me/ns#"
      xmlns:fb="http://www.facebook.com/2008/fbml">
<HEAD>
    <title>%s</title>
    <!-- Global site tag (gtag.js) - Google Analytics -->
    <script async src="https://www.googletagmanager.com/gtag/js?id=UA-108858221-1"></script>
    <script>
      window.dataLayer = window.dataLayer || [];
      function gtag(){dataLayer.push(arguments);}
      gtag('js', new Date());

      gtag('config', 'UA-108858221-1');
    </script>
    <meta http-equiv="Content-type" content="text/html;charset=UTF-8"/>
    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0" />
    <meta name="apple-mobile-web-app-capable" content="no" />
    <link rel="apple-touch-icon" href="%simages/icon_300.png"/>
    <meta property="og:type" content="website"/>
    <meta property="og:title" content="%s"/>
    <meta property="og:description" content="%s"/>
    <meta property="og:image" content="%simages/thefuture.png"/>

    <meta name="twitter:card" content="summary_large_image">
    <meta name="twitter:site" content="@SuffolkLITLab">
    <meta name="twitter:creator" content="@SuffolkLITLab">
    <meta name="twitter:title" content="%s">
    <meta name="twitter:description" content="%s">
    <meta name="twitter:image" content="http://www.davidcolarusso.com/images/thefuture.png"/>

    <meta http-equiv="X-UA-Compatible" content="IE=edge" />
    <link rel="shortcut icon" type="image/x-icon" href="/favicon.ico">
    <link rel="apple-touch-icon" href="%s/images/icon.png"/>
    <link rel="stylesheet" href="https://code.jquery.com/ui/1.11.1/themes/smoothness/jquery-ui.css">
    <link rel="stylesheet" type="text/css" href="%s/css/style.css?v=2">
    <script src="https://code.jquery.com/jquery-1.11.1.min.js"></script>
    <script src="https://code.jquery.com/jquery-1.10.2.js"></script>
    <script src="https://code.jquery.com/ui/1.11.1/jquery-ui.js"></script>

</HEAD>
<BODY BGCOLOR="#ffffff" BACKGROUND="" MARGINWIDTH="0" MARGINHEIGHT="0">
<div style="box-sizing: border-box;width:100%%;background:#d6392b;color:white;padding:15px;text-align:center;"><b>This page is not active.</b> This page is just for testing purposes. <b>DO NOT RELY ON THE CONTENT FOUND HERE!</b></div>
<div class="%s">

    <div id="icon" style="background-size: 100px 100px;background-image: url('%simages/seal.jpg');"><a href="." alt="home"><img src="%simages/space.gif" width="100px" height="100px;" border="0" alt="LIT Logo"/></a></div>
    <h1 style="text-align:center;">
        The Legal Innovation & Technology Lab's Form Explorer <sup><font size=+1>Beta</font></sup>
        <center style="margin-top:5px;">
          <span class="subtitle">@ Suffolk Law School&nbsp;<font style="font-weight:normal">Last updated %s</font></span>
        </center>
    </h1>
"""%(title,path,title,description,path,title,description,path,path,pclass,path,path,today)
    return html
    
def footer(path="./"):
    html = """

<div class="footer">
    <a href="/" alt="home"><img src="%simages/blue_logo.png" width="50px" align="left" border="0" alt="LIT Logo"/></a>
<font size=-1><a href="mailto:litlab@suffolk.edu">Email</a> | <a href="https://github.com/SuffolkLITLab" target="_blank">GitHub</a> | <a href="/terms">Terms &amp; Privacy</a> | <a href="/credits">Credits</a></font>
</div>

</BODY>
</HTML>
"""%(path)
    
    return html

# Home page

In [200]:
def home():
    html = """
    <div class="menu_bar">
        <p style="text-align:center;">
      <a href="https://suffolklitlab.org/" class="menu">&nbsp;LIT Lab Home&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      <a href="." class="menu" style="color:black;">&nbsp;About&nbsp;The&nbsp;Explorer&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      &nbsp;&nbsp;<span style="font-family: sans-serif;font-size: 12px;text-transform: uppercase;">Explore:</span>&nbsp;&nbsp;
            <a href="sim/" class="menu">&nbsp;Similar Forms&nbsp;</a>
            <a href="flow/" class="menu">&nbsp;Flows&nbsp;</a>
      <a href="list/" class="menu">&nbsp;Lists&nbsp;</a>
        </p>
    </div>

  <div class="content"><h2><a name="what" href="#what" class="anchor"></a>What is the Form Explorer?</h2>

    <div style="float:left;width:100%">
      <div class="r_img_embed" style="max-width:500px;">
        <img src="images//bigdata.png" alt="Image of the character Data as a giant standing next to the Supreme Court" width="100%"/>
        <div class="caption">Big Data &amp; the Law, h/t <a href="https://flic.kr/p/7YZubW" target="_blank">Tim Sackton</a> &amp; <a href="https://medium.com/@wtrsld/big-data-made-me-do-it-5bfc3f46871c" target="_blank">Josh Lee</a></div>
      </div>
      <p>
        Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam non erat quis augue mollis convallis. Aliquam eu varius massa. Nam convallis lacinia magna, eget aliquet orci congue at. Interdum et malesuada fames ac ante ipsum primis in faucibus. Curabitur eu nisl eu libero porttitor convallis quis ut est. Duis porttitor turpis erat, ut viverra metus fringilla sit amet. Vestibulum tincidunt nibh id libero euismod, sed congue diam lobortis. Vestibulum maximus posuere odio, eget placerat nunc efficitur quis. Proin ut odio sem. Phasellus interdum massa sed sodales eleifend. Cras condimentum, risus et volutpat tincidunt, elit justo vulputate lacus, eget hendrerit purus turpis in nisl. Fusce placerat ex ac hendrerit condimentum. Pellentesque eget auctor leo.
      </p><p>
        Vivamus porttitor posuere sodales. Maecenas id pharetra diam. Nam quis erat quis enim placerat congue et non dui. Donec tempor vehicula ante eget tempor. Pellentesque habitant morbi tristique senectus et netus et malesuada fames ac turpis egestas. Sed vitae orci neque. Maecenas ultricies dignissim neque quis consectetur. In sit amet luctus erat. Pellentesque ultrices sed dui nec feugiat. In nunc ante, bibendum non molestie imperdiet, molestie vel massa. Vestibulum a dictum orci, fringilla rutrum quam. Cras pellentesque congue porttitor. Ut neque felis, tincidunt euismod porttitor sed, feugiat et ligula. Duis vehicula eleifend elit sit amet tempus. Vestibulum condimentum libero vitae justo accumsan, vel sodales erat mattis.
      </p><p>
        Nam nec turpis elementum, tempus massa eget, dictum dui. Donec rutrum nisi pretium faucibus vestibulum. Mauris in ultrices arcu. Nulla vel sem a velit pretium consequat sed vitae felis. Nunc velit tortor, varius at accumsan id, maximus at velit. In cursus metus a purus condimentum convallis. Donec ullamcorper leo eget massa iaculis vehicula. Fusce bibendum aliquam arcu semper laoreet. Quisque consectetur finibus purus sed faucibus. Sed purus ligula, malesuada et diam quis, tincidunt suscipit enim. Curabitur eget quam a tellus suscipit sollicitudin nec ut arcu. Donec bibendum urna a dapibus porttitor. Aliquam in iaculis orci. Etiam condimentum auctor elit eget aliquam. Nulla facilisi. Proin at vestibulum nunc.
      </p>
    </div>
"""    
    return html

In [201]:
html = header("Form Explorer","description goes here")+home()+footer()
text_file = open("docs/index.html", "w")
text_file.write(html)
text_file.close()

# redirects

In [202]:
def redirects(jur):
    html = """<html>
<head>
<META http-equiv="CACHE-CONTROL" CONTENT="NO-CACHE">
<meta http-equiv="refresh" content="0; url=%s" />
</head>
</html>
"""%jur
    return html

In [203]:
html = redirects(default_jur)

text_file = open("docs/sim/index.html", "w")
text_file.write(html)
text_file.close()

text_file = open("docs/flow/index.html", "w")
text_file.write(html)
text_file.close()

text_file = open("docs/list/index.html", "w")
text_file.write(html)
text_file.close()

# Similarity Viewer

In [204]:
def simview(jur,path="./"):
    html = """
    <div class="menu_bar">
        <p style="text-align:center;">
      <a href="https://suffolklitlab.org/" class="menu">&nbsp;LIT Lab Home&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      <a href="../../" class="menu">&nbsp;About&nbsp;The&nbsp;Explorer&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      &nbsp;&nbsp;<span style="font-family: sans-serif;font-size: 12px;text-transform: uppercase;">Explore:</span>&nbsp;&nbsp; <select style="width:100px;" onChange="window.location.href='../'+this.value">"""
    
    for item in jurs: 
        if item[0]==jur[0]:
            selected = "SELECTED"
        else:
            selected = ""
        html += "<option value=\"%s\" %s>%s</option>"%(item[0],selected,item[1])
        
    html += """
      </select>&nbsp;&nbsp;
            <a href="../../sim/%s" class="menu" style="color:black;">&nbsp;Similar Forms&nbsp;</a>
            <a href="../../flow/%s" class="menu">&nbsp;Flows&nbsp;</a>
      <a href="../../list/%s" class="menu">&nbsp;Lists&nbsp;</a>
        </p>
    </div>

    <div class="content_wide"><h2><a name="what" href="#what" class="anchor"></a>Similar Forms View (%s Court Forms)</h2>
        <p>
          Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam non erat quis augue mollis convallis. Aliquam eu varius massa. Nam convallis lacinia magna, eget aliquet orci congue at. Interdum et malesuada fames ac ante ipsum primis in faucibus. Curabitur eu nisl eu libero porttitor convallis quis ut est. Duis porttitor turpis erat, ut viverra metus fringilla sit amet. Vestibulum tincidunt nibh id libero euismod, sed congue diam lobortis. Vestibulum maximus posuere odio, eget placerat nunc efficitur quis. Proin ut odio sem. Phasellus interdum massa sed sodales eleifend. Cras condimentum, risus et volutpat tincidunt, elit justo vulputate lacus, eget hendrerit purus turpis in nisl. Fusce placerat ex ac hendrerit condimentum. Pellentesque eget auctor leo.
        </p>
        <!--
            Possible library for viewer:
            https://www.sigmajs.org/
        <iframe src="https://www.sigmajs.org/demo/index.html" width="100%%" height="700px;"></iframe>
        -->

    </div>
"""%(jur[0],jur[0],jur[0],jur[1])
    return html

In [205]:
for jur in jurs:    
    print(jur)
    path = "../../"
    
    if not os.path.exists("docs/sim/%s"%(jur[0])):
        os.makedirs("docs/sim/%s"%(jur[0]))
        
    html = header("Form Explorer","description goes here",path,wide=1)+simview(jur,path)+footer(path)
    text_file = open("docs/sim/%s/index.html"%jur[0], "w")
    text_file.write(html)
    text_file.close()

['MI', 'Michigan']
['UT', 'Utah']


# Flow View

In [206]:
def flowview(jur,path="./"):
    html = """
    <div class="menu_bar">
        <p style="text-align:center;">
      <a href="https://suffolklitlab.org/" class="menu">&nbsp;LIT Lab Home&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      <a href="../../" class="menu">&nbsp;About&nbsp;The&nbsp;Explorer&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      &nbsp;&nbsp;<span style="font-family: sans-serif;font-size: 12px;text-transform: uppercase;">Explore:</span>&nbsp;&nbsp; <select style="width:100px;" onChange="window.location.href='../'+this.value">"""
    
    for item in jurs: 
        if item[0]==jur[0]:
            selected = "SELECTED"
        else:
            selected = ""
        html += "<option value=\"%s\" %s>%s</option>"%(item[0],selected,item[1])
        
    html += """
      </select>&nbsp;&nbsp;
            <a href="../../sim/%s" class="menu">&nbsp;Similar Forms&nbsp;</a>
            <a href="../../flow/%s" class="menu" style="color:black;">&nbsp;Flows&nbsp;</a>
      <a href="../../list/%s" class="menu">&nbsp;Lists&nbsp;</a>
        </p>
    </div>


      <div class="content_wide"><h2><a name="what" href="#what" class="anchor"></a>User Flow (%s Court Forms)</h2>
        <p>
          Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam non erat quis augue mollis convallis. Aliquam eu varius massa. Nam convallis lacinia magna, eget aliquet orci congue at. Interdum et malesuada fames ac ante ipsum primis in faucibus. Curabitur eu nisl eu libero porttitor convallis quis ut est. Duis porttitor turpis erat, ut viverra metus fringilla sit amet. Vestibulum tincidunt nibh id libero euismod, sed congue diam lobortis. Vestibulum maximus posuere odio, eget placerat nunc efficitur quis. Proin ut odio sem. Phasellus interdum massa sed sodales eleifend. Cras condimentum, risus et volutpat tincidunt, elit justo vulputate lacus, eget hendrerit purus turpis in nisl. Fusce placerat ex ac hendrerit condimentum. Pellentesque eget auctor leo.
        </p>
        <div style="overflow-x: scroll;:hidden;overflow-y: hidden;">
          <img src="../../images/.jpg" alt="" width="1500px"/>
        </div>

      </div>
"""%(jur[0],jur[0],jur[0],jur[1])
    return html

In [207]:
for jur in jurs:    
    print(jur)
    path = "../../"
    
    if not os.path.exists("docs/flow/%s"%(jur[0])):
        os.makedirs("docs/flow/%s"%(jur[0]))
        
    html = header("Form Explorer","description goes here",path,wide=1)+flowview(jur,path)+footer(path)
    text_file = open("docs/flow/%s/index.html"%jur[0], "w")
    text_file.write(html)
    text_file.close()

['MI', 'Michigan']
['UT', 'Utah']


# List VIew

In [208]:
def listview(jur,path="./"):
    html = """
    <div class="menu_bar">
        <p style="text-align:center;">
      <a href="https://suffolklitlab.org/" class="menu">&nbsp;LIT Lab Home&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      <a href="../../" class="menu">&nbsp;About&nbsp;The&nbsp;Explorer&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      &nbsp;&nbsp;<span style="font-family: sans-serif;font-size: 12px;text-transform: uppercase;">Explore:</span>&nbsp;&nbsp; <select style="width:100px;" onChange="window.location.href='../'+this.value">"""
    
    for item in jurs: 
        if item[0]==jur[0]:
            selected = "SELECTED"
        else:
            selected = ""
        html += "<option value=\"%s\" %s>%s</option>"%(item[0],selected,item[1])
        
    html += """
      </select>&nbsp;&nbsp;
            <a href="../../sim/%s" class="menu">&nbsp;Similar Forms&nbsp;</a>
            <a href="../../flow/%s" class="menu">&nbsp;Flows&nbsp;</a>
      <a href="../../list/%s" class="menu" style="color:black;">&nbsp;Lists&nbsp;</a>
        </p>
    </div>
"""%(jur[0],jur[0],jur[0])
    
    html += """
    <div class="content"><h2><a name="what" href="#what" class="anchor"></a>List of %s Court Forms</h2>

    <p>
      Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam non erat quis augue mollis convallis. Aliquam eu varius massa. Nam convallis lacinia magna, eget aliquet orci congue at. Interdum et malesuada fames ac ante ipsum primis in faucibus. Curabitur eu nisl eu libero porttitor convallis quis ut est.
    </p>"""%(jur[1])

    for cat in files_df[files_df["jurisdiction"]==jur[0]]["group"].unique():
        html += """
        <h3><a name="%s" href="#%s" class="anchor"></a>%s</h3>
        <ul>"""%(cat,cat,cat)

        for index,row in files_df[(files_df["jurisdiction"]==jur[0]) & (files_df["group"]==cat)].iterrows():
            #print(row[1][0])
            html += """<li><a href="../../form/%s/%s.html">%s</a></li>"""%(jur[0],row["id"],row["title"])
            
        html += "</ul>"

    return html

In [209]:
for jur in jurs:    
    print(jur)
    path = "../../"
    
    if not os.path.exists("docs/list/%s"%(jur[0])):
        os.makedirs("docs/list/%s"%(jur[0]))
        
    html = header("Form Explorer","description goes here",path,wide=0)+listview(jur,path)+footer(path)
    text_file = open("docs/list/%s/index.html"%(jur[0]), "w", encoding="utf-8")
    text_file.write(html)
    text_file.close()

['MI', 'Michigan']
['UT', 'Utah']


# Create form pages

In [210]:
def formview(jur,path="./"):
    html = """
    <div class="menu_bar">
        <p style="text-align:center;">
      <a href="https://suffolklitlab.org/" class="menu">&nbsp;LIT Lab Home&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      <a href="../../" class="menu">&nbsp;About&nbsp;The&nbsp;Explorer&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      &nbsp;&nbsp;Explore: """
    
    
    for item in jurs: 
        if item[0]==jur[0]:
            html += "%s"%item[1]

    html += """
      &nbsp;&nbsp;
            <a href="../../sim/%s" class="menu">&nbsp;Similar Forms&nbsp;</a>
            <a href="../../flow/%s" class="menu">&nbsp;Flows&nbsp;</a>
      <a href="../../list/%s" class="menu">&nbsp;Lists&nbsp;</a>
        </p>
    </div>

"""%(jur[0],jur[0],jur[0])
    
    html += """
  <div class="content_wide" style="padding-left:0px;">
    <div style="float:left;width:60%%;margin-right:20px;box-sizing: border-box;">
      <iframe src="../../forms/%s.pdf" width="100%%" height="1100px;"></iframe>
    </div>
    <div style="float:left;width:38%%;box-sizing: border-box;padding:0;margin:0;">
      <h1><a name="name" href="#name" class="anchor"></a>%s</h1>
      <p>
        Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam non erat quis augue mollis convallis. Aliquam eu varius massa. Nam convallis lacinia magna, eget aliquet orci congue at. Interdum et malesuada fames ac ante ipsum primis in faucibus. Curabitur eu nisl eu libero porttitor convallis quis ut est.
      </p>
      <p>
        <b>Downloads:</b> In order to get the data on this page we scan forms for... You can download both the <a href="../form/">original form</a> and the <a href="../form/">machine-processed form</a> with normalized data fields.
      </p>
      <h3><a name="cat1" href="#cat1" class="anchor"></a>About This Form:</h3>
      <ul>"""%(row["id"],row["title"])
    
    html += """
        <li>Sourced from: <a href="" target="_blank">UT Courts</a> (2021-10-19)</li>
        <li>X pages long</li>
        <li>Y questions per page, on average</li>
        <li>Reading Level: Y</li>
        <li>Form Type: Family (best guess)</li>
      </ul>
      <h3><a name="cat1" href="#cat1" class="anchor"></a>About Users of This Form:*</h3>
      <p>Filers of this form file an average of <b>X</b> other forms. <b>Y%</b> of them identify as male.</p>
      <h3><a name="cat1" href="#cat1" class="anchor"></a>Identified Data Fields and Utilization:<sup>&dagger;</sup></h3>
      <ul>
        <li>P1 <a href="../form/">Name</a> 100% completed. <a href="">Learn more about distribution of answers.</a></li>
        <li>P1 <a href="../form/">Address</a> 100% completed. <a href="">Learn more about distribution of answers.</a></li>
        <li><a href="../form/">show more</a></li>
      </ul>

      </ul>
      <h3><a name="cat1" href="#cat1" class="anchor"></a>Similar forms:</h3>
      <p>For more context, visit the <a href="../sim/">Similar Forms View</a>.</p>
      <ul>
        <li><a href="../form/">Form one</a> (shares 90% of data fields)</li>
        <li><a href="../form/">Form one</a> (shares 70% of data fields)</li>
        <li><a href="../form/">show more</a></li>
      </ul>
      <h3><a name="cat1" href="#cat1" class="anchor"></a>Before using this form, users used:*</h3>
      <p>For more context, visit the <a href="../flow/">Flows View</a>.</p>
      <ul>
        <li><a href="../form/">Form one</a> (shares 90% of data fields)</li>
        <li><a href="../form/">Form one</a> (shares 70% of data fields)</li>
        <li><a href="../form/">show more</a></li>
      </ul>
      <h3><a name="cat1" href="#cat1" class="anchor"></a>After using this form, users used:*</h3>
      <p>For more context, visit the <a href="../flow/">Flows View</a>.</p>
      <ul>
        <li><a href="../form/">Form one</a> (shares 90% of data fields)</li>
        <li><a href="../form/">Form one</a> (shares 70% of data fields)</li>
        <li><a href="../form/">show more</a></li>
      </ul>
      <h3><a name="cat1" href="#cat1" class="anchor"></a>Create an Interactive Online Version of this Form:</h3>
      <p>Using the Document Assembly Line Weaver you can...</p>
      <p>
        <a href="">Run this form through the Weaver</a> (How To)
      </p>

    </div>
  </div>
    """
    return html

In [211]:
for jur in jurs:    
    if not os.path.exists("docs/form/%s"%(jur[0])):
        os.makedirs("docs/form/%s"%(jur[0]))

for index,row in files_df.iterrows():
    print(row["title"])
    
    html = header("Form Explorer","description goes here",path,wide=1)+formview([row["jurisdiction"],""],path)+footer(path)
    text_file = open("docs/form/%s/%s.html"%(row["jurisdiction"],row["id"]), "w", encoding="utf-8")
    
    text_file.write(html)
    text_file.close()

Community Service Worksheet Third District Juvenile Court  
Third District Juvenile Court Work Program Referral 
Utah State District Juvenile Court Probation Order 
Adoptee's Consent to Adoption and Waiver of Rights 
Adoption Agreement 
Adoption Decree 
Birth Mother's Consent to Adoption and Waiver of Rights 
Certificate of Readiness for Adoption Hearing 
Ex Parte Motion for Alternative Service 
Findings of Fact and Conclusions of Law 
Answer to Complaint to Recover Possession of Property
Addendum A to Verified Declaration About Paternity (Father's Name and Whereabouts Known)
14-Day Notice, Traffic
Motion and Order for Discharge From Probation
Bond, Claim and Delivery
Ex Parte Petition Regarding Voluntary Foster Care Agreement
Affidavit Concerning Financial Status- Leave to Appeal Granted by Supreme Court
Final Statement on Garnishment of Periodic Payments
Acceptance of Receivership Appointment (Receivership Estate)
Application and Ex Parte Order to File Lien on Real Property Subject t